In [2]:
import os
import numpy as np
import pandas as pd
import json
from copy import deepcopy
import re
# -*- coding:utf-8 -*- 

def mk_dir(file_path):

    folder = os.path.exists(file_path)
    if not folder:
        os.makedirs(file_path)

# QA训练集

In [4]:
col_ll = [["样本量","num",{"1":"How many patients are ivolved in this study?"}]
["患者性别","gender",{"1":"What's gender of the patient?"}]
["年龄","age",{"1":"How old is patient?"}]
["证据等级","evi",{"1":"Where is pathogen isolated?"}]
["免疫状态","immu",{"1":"What's patients immunity state?"}]]

file_dir_l = {"Virology", "Mycology"}
for file_dir in file_dir_l:
    for col_l in col_ll:
        col_name = col_l[0]
        col_ab = col_l[1]
        que_d = col_l[2]

In [3]:
sped_path = "../5.Orgnize/文献标注-校对/"
sped_l = [ele for ele in os.listdir(sped_path) if col_ab in ele]
txt_path = "../4.Label_prepare/%s/" % file_dir
train_qa = []

for sped in sped_l:

    # 读取标注信息
    print(sped)
    df = pd.read_excel("%s/%s" % (sped_path, sped), header=0)
    df = df.loc[df["是否是致病菌"] == "是",:]

    for idx in df.index.tolist():

        # 创建WHAT Question样本
        if np.isnan(df.loc[idx,"old_start" ]) == False:
            if type(df.loc[idx,"answer" ]) == float:
                df.loc[idx,"answer" ] = "NaN"
                
            if df.loc[idx, "old_start" ] < len(df.loc[idx, "full"]):

                if df.loc[idx, "answer"][0] == df.loc[idx, "full"].replace("\n","")[int(df.loc[idx, "old_start" ])]:
                    k = int(df.loc[idx, "old_start" ])
                else:
                    k = int(df.loc[idx, "old_start" ]) + 1

                qa = {
                    "qas": [
                        {
                            "id": str(df.loc[idx, "ID"]),
                            "question": que_d["1"],
                            "answers": [
                                {
                                    "text": df.loc[idx, "answer" ],
                                    "answer_start": k,
                                }
                            ]
                        }
                    ],
                    "context": df.loc[idx, "full"].replace("\n","")}
                train_qa.append(qa)
                
    mk_dir("input/input_%s" % col_ab)
    df.to_csv("input/input_%s/%s.csv" % (col_ab, sped.split("_")[0]), header=True, index=False)

train_set = {
    "data": [{
        'paragraphs': train_qa,
        'title': "LungB"
    }],
    "version": "LungB"
}
mk_dir("input/input_%s" % col_ab)
json.dump(train_set, open("input/input_%s/train_set.json" % (col_ab), "w"))


Anaerobic_gender.xls
Curved_gender.xls
GramN_gender.xls
GramPc_gender.xls
GramPr_gender.xls
Intracellular_gender.xls


# QA预测集（现在主要应用）

In [ ]:
# 加载判断了的病原菌信息
prediction = json.load(open("output/output_pat/%s/predictions_.json" % file_dir, 'r'))
result = {}
for i in prediction.keys():
    toke = i.split(":")[0]
    species = ""
    for j in toke.split("_")[0:-1]:
        species = species + " " + j
    ncbi = "https://pubmed.ncbi.nlm.nih.gov/%s/" % toke.split("_")[-1]

    result[toke] = [toke.split("_")[-1], species, prediction[i][1][0], ncbi]
result = pd.DataFrame(result, index = ["ID", "SPECIES", "rate", "link"]).T

In [6]:
# 根据上述筛选出的样本的数据集
file_dir_l = {"Virology"}

col_ll = [["样本量","num",{"1":"How many patients are ivolved in this study?"}],
["患者性别","gender",{"1":"What's gender of the patient?"}],
["年龄","age",{"1":"How old is patient?"}],
["证据等级","evi",{"1":"Where is pathogen isolated?"}],
["免疫状态","immu",{"1":"What's patients immunity state?"}]]

for file_dir in file_dir_l:
    prediction = json.load(open("output/output_pat/%s/predictions_.json" % file_dir, 'r'))
    result = {}
    for i in prediction.keys():
        toke = i.split(":")[0]
        species = ""
        for j in toke.split("_")[0:-1]:
            species = species + " " + j
        ncbi = "https://pubmed.ncbi.nlm.nih.gov/%s/" % toke.split("_")[-1]

        result[toke] = [toke.split("_")[-1], species, prediction[i][1][0], ncbi]
    result = pd.DataFrame(result, index = ["ID", "SPECIES", "rate", "link"]).T
    
    for col_l in col_ll:
        col_name = col_l[0]
        col_ab = col_l[1]
        que_d = col_l[2]

        sped_path = "../4.Label_prepare/%s" % file_dir

        sped_l = [ele for ele in os.listdir(sped_path) if '.' not in ele]

        test_qa = []

        for sped in sped_l:
            
            df = pd.DataFrame(np.zeros((0,2)), columns = ["SPECIES", "full"])
            
            id_l = [ele for ele in os.listdir("%s/%s" % (sped_path, sped)) if 'txt' in ele]
            id_l = [ele for ele in id_l if ele.replace(".txt", "") in result.index]

            for idx in id_l:

                if result.loc[idx.replace(".txt", ""), "rate"] > 0.5:
                
                    with open("%s/%s/%s" % (sped_path, sped, idx)) as f:
                        txt0 = f.readlines()
                    with open("%s/%s/%s" % (sped_path, sped, idx)) as f:
                        txt = f.read()
                    
                    idx = idx.split(".")[0]
                    df.loc[idx] = [txt0[0].split(":")[1], txt.replace("\n", "")]
                    
                    qa = {'qas': [{
                        'id': '%s:%s' % (idx, df.loc[idx, "SPECIES"]),
                        'question': que_d["1"],
                        "answers": []}],
                        'context': '%s' % df.loc[idx, "full"]}
                    test_qa.append(qa)
            
            mk_dir("input/input_%s/test" % (col_ab))
            df.to_csv("input/input_%s/test/%s.csv" % (col_ab, sped), header=True, index=True)

        test_set = {"data":[{'paragraphs':test_qa, 'title':"LungB"}], "version":"LungB"}

        mk_dir("input/input_%s/%s" % (col_ab, file_dir))
        json.dump(test_set, open("input/input_%s/%s/test_set.json" % (col_ab, file_dir), "w"))